In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
RATIO_THRESHOLD = 85 # filter out the nouns whose gender-ratio is below this value
MAX_SUFFIX_LENGTH = 4 # the analysis is carried out on the suffixes from 1 to this value (included)
ROW_COUNT = 100 # number of suffixes to display

In [3]:
PATH_OR_URL = "https://raw.githubusercontent.com/laowantong/Google-French-Nouns-with-Genders/master/french_nouns.tsv"
df = pd.read_csv(PATH_OR_URL, sep="\t", encoding="utf8")

# Calculating the results

In [4]:
def group_by_suffix_length(df, n):
    """Calculate a table of frequencies indexed by total count, suffix and gender."""
    
    # First step. Create a left table similar to (example for n == 3):
    #
    #               freq                                                     nouns  
    # noun gender     
    # ...       
    # ace  f      6572736 (place, surface, menace, face, race, trace, préface, ...  
    #      m      1034050 (espace, face, place, menace, cyberespace, palace, ra...  
    # ach  m        23832 (krach, coach, almanach, midrach)  
    # ack  m        14987 (feedback, crack, pack, black, snack, biofeedback, back)  
    # ...
    #
    # NB: An erroneous feminine gender is assigned to "face", "place", and "menace".
    #     This corresponds to relatively high-frequency errors in the Google digram dataset.
    
    df["count_and_noun"] = df[["count", "noun"]].apply(tuple, axis=1) # helper column for the sort
    left = \
        df[
            df["noun"].str.len() >= n # filter out any word shorter than the desired suffix length
        ].groupby([
            df["noun"].str.slice(-n), # group by both suffix
            "gender"                  #   and gender
        ]).agg({
            "count": "sum",             # totalize the frequencies for each couple (noun, gender)
            "count_and_noun": lambda l: # transform each list of count and noun sharing both suffix and gender
                tuple( #                  into a tuple of
                    item[1] for item in sorted(l, reverse=True) # nouns sorted by decreasing frequency
                ),
        })
    left.columns = ["freq", "nouns"] # rename the resulting columns
    df.drop(["count_and_noun"], axis=1, inplace=True) # restore df in its original state
    
    # Second step. Create a right table similar to:
    #
    #          total
    # noun          
    # ...
    # ace    7606786
    # ach      23832
    # ack      14987
    # ...

    right = \
        df[
            df["noun"].str.len() >= n # filter out any word shorter than the desired suffix length
        ].groupby(
            df["noun"].str.slice(-n)  # group by suffix
        ).agg({
            "count": "sum" # totalize the frequencies for each noun
        })
    right.columns = ["total"]

    # Final step. Join the left and right tables on column "noun".
    #
    #                         freq                                                    nouns
    # total   suffix gender          
    # 7606786 ace    f       86.41 (place, surface, menace, face, race, trace, préface, ...  
    #                m       13.59 (espace, face, place, menace, cyberespace, palace, ra...  
    # 23832   ach    m      100.00 (krach, coach, almanach, midrach)    
    # 14987   ack    m      100.00 (feedback, crack, pack, black, snack, biofeedback, back)  
    
    result = pd.merge(
            left.reset_index(),  # transform the indexes into columns
            right.reset_index(), #   to make them possible to join
            on=["noun"]
        ).set_index(["total", "noun", "gender"])       # transform these columns back into indexes
    result.index.names = ["total", "suffix", "gender"] # and rename the second one
    result["freq"] = 100 * result["freq"] / result.index.get_level_values("total") # frequencies as percentages

    return result

In [5]:
def collect_all_data(df):
    """Calculate a table with ALL suffixes between 1 and MAX_SUFFIX_LENGTH."""
    
    # Call the previous function for all the desired suffix lengths
    #   and concatenate the results in a single table
    result = pd.DataFrame()
    for i in range(1, MAX_SUFFIX_LENGTH + 1):
        result = result.append(group_by_suffix_length(df, i))
    
    # Upgrade temporarily two indexes to columns to make them possible to sort
    result["total_from_index"] = result.index.get_level_values("total")
    result["suffix_from_index"] = result.index.get_level_values("suffix")
    
    # Sort by total, suffix and gender frequency
    result.sort_values(by=["total_from_index", "suffix_from_index", "freq"], inplace=True, ascending=False)
    
    # Drop the temporary columns
    result.drop(["total_from_index", "suffix_from_index"], axis=1, inplace=True)
    
    # Drop the "total" index, which is useless since the table is now sorted by total first
    result.reset_index(level=0, drop=True, inplace=True)
    
    # The result is ordered by decreasing suffix frequencies, suffixes and gender frequencies:
    #
    #                 freq                                                           nouns
    # suffix gender          
    # e      f       70.79  (vie, politique, guerre, ville, nature, forme, mise, recher... 
    #        m       29.21  (monde, rôle, système, cadre, nombre, problème, texte, doma... 
    # n      f       86.63  (fin, question, situation, population, production, région, ... 
    #        m       13.37  (plan, terrain, moyen, lien, roman, an, chemin, besoin, len... 
    # on     f       96.74  (question, situation, population, production, région, créat... 
    #        m        3.26  (ton, don, garçon, bouton, million, son, patron, échantillo... 
    # ion    f       99.65  (question, situation, population, production, région, créat... 
    #        m        0.35  (million, avion, camion, lion, champion, bastion, pion, esp... 
    # tion   f       99.97  (question, situation, population, production, création, for... 
    #        m        0.03  (bastion, cation, fonction, question, situation, direction,...
    # ...
    
    return result

In [6]:
def best_suffixes_to_learn(df, how_many=None):
    """Eliminate the suffixes whose gender repartition is not "interesting" enough."""
    
    result = pd.DataFrame()
    seen = dict()
    df = collect_all_data(df)
    
    # Iterate on the suffixes (sorted by decreasing frequency)
    for suffix in df.index.get_level_values(0).unique(): # iterate on the suffixes
        
        # Create a small dataframe consisting only in the rows of the current suffix
        current_group = df.xs(suffix, drop_level=False)
        
        # Extract the row corresponding to the most common gender for this suffix
        group_first_row = current_group.head(1)["freq"]
        gender = group_first_row.index.values[0][0] # retrieve the gender
        freq = group_first_row.item()               # and its frequency
        
        # Keep "interesting" suffixes
        if freq > seen.get(suffix[1:], # if the suffix does not end with a smaller suffix already kept
                           RATIO_THRESHOLD # or has a frequency greater than the RATIO_THRESHOLD
                  ) + len(suffix) - 1: # plus a certain amount depending of its length - 1
            seen[suffix] = freq # keep it
            result = result.append(current_group)
            if how_many and len(seen) > how_many:
                break # stop after a given number of rows in the result
    
    return result

In [7]:
# Calculate the ROW_COUNT first interesting rows
best = best_suffixes_to_learn(df, ROW_COUNT)

# Displaying the results

In [8]:
pd.options.display.float_format = "{:,.2f}".format # two decimals ought to be enough for anybody
pd.options.display.max_colwidth = 100 # longer lists of nouns in each row

In [9]:
def show(*suffixes):
    """A helper function displaying some or all rows of the resulting table."""
    try:
        if suffixes:
            display(best.ix[list(suffixes)])
        else:
            with pd.option_context('display.max_rows', len(best)):
                display(best.iloc[:len(best)])
    except KeyError:
        print "Missing suffix! You may decrease RATIO_THRESHOLD and/or increase ROW_COUNT."

## Displaying one given suffix

In [10]:
show("age")

freq  \
suffix gender         
age    m      90.18   
       f       9.82   

                                                                                                             nouns  
suffix gender                                                                                                       
age    m       (passage, langage, personnage, mariage, village, visage, message, voyage, paysage, ouvrage, chôm...  
       f       (image, page, plage, rage, cage, sage, nage, partage, sauvage, langage, passage, mariage, dévisa...

## Displaying several given suffixes

In [11]:
show("ment", "ure")

freq  \
suffix gender         
ment   m      99.93   
       f       0.07   
ure    f      99.58   
       m       0.42   

                                                                                                             nouns  
suffix gender                                                                                                       
ment   m       (développement, gouvernement, mouvement, moment, sentiment, traitement, changement, comportement...  
       f       (jument, recherchedéveloppement, développement, gouvernement, ment, transforment, mouvement, lég...  
ure    f       (nature, culture, mesure, structure, littérature, lecture, figure, procédure, peinture, rupture,...  
       m       (murmure, mercure, chlorure, mesure, jure, rassure, centaure, demeure, parjure, sulfure, figure,...

## Displaying all suffixes

In [12]:
show()

freq  \
suffix gender          
n      f       86.63   
       m       13.37   
on     f       96.74   
       m        3.26   
ion    f       99.65   
       m        0.35   
t      m       88.20   
       f       11.80   
é      f       86.93   
       m       13.07   
té     f       96.35   
       m        3.65   
ce     f       90.49   
       m        9.51   
nt     m       99.51   
       f        0.49   
ie     f       99.10   
       m        0.90   
nce    f       97.24   
       m        2.76   
se     f       98.38   
       m        1.62   
ment   m       99.93   
       f        0.07   
ure    f       99.58   
       m        0.42   
l      m       99.52   
       f        0.48   
lité   f      100.00   
       m        0.00   
ique   f       93.83   
       m        6.17   
u      m       94.07   
       f        5.93   
age    m       90.18   
       f        9.82   
lle    f       98.59   
       m        1.41   
ée     f       97.01   
       m        2.99   
che    f       93.97   
       m        6.03   
teur   m       94.16   
       f        5.84   
et     m       99.87   
       f        0.13   
ière   f       98.50   
       m        1.50   
er     m       86.05   
       f       13.95   
ème    m       98.22   
       f        1.78   
part   f       92.96   
       m        7.04   
ir     m       95.01   
       f        4.99   
rité   f       99.97   
       m        0.03   
eau    m       94.03   
       f        5.97   
sme    m       99.89   
       f        0.11   
été    f       99.71   
       m        0.29   
at     m       99.91   
       f        0.09   
rre    f       95.14   
       m        4.86   
onde   m       89.25   
       f       10.75   
esse   f       99.84   
       m        0.16   
ison   f       99.21   
       m        0.79   
as     m       99.58   
       f        0.42   
ps     m       99.99   
       f        0.01   
fois   f       99.95   
       m        0.05   
an     m       99.08   
       f        0.92   
loi    f       93.42   
       m        6.58   
en     m       99.62   
       f        0.38   
rte    f       97.01   
       m        2.99   
d      m       95.93   
       f        4.07   
ôle    m       98.99   
       f        1.01   
orme   f       99.35   
       m        0.65   
fin    f       99.30   
       m        0.70   
ier    m       99.86   
       f        0.14   
rise   f       99.76   
       m        0.24   
ut     m       99.91   
       f        0.09   
ait    m       99.06   
       f        0.94   
oir    m       99.86   
       f        0.14   
eu     m       99.97   
       f        0.03   
m      m       95.40   
       f        4.60   
rs     m       99.85   
       f        0.15   
dre    m       95.51   
       f        4.49   
us     m       92.56   
       f        7.44   
ogie   f      100.00   
rche   f       98.25   
       m        1.75   
tte    f       98.52   
       m        1.48   
xte    m       99.92   
       f        0.08   
emps   m       99.98   
       f        0.02   
leur   f       91.38   
       m        8.62   
nte    f       95.92   
       m        4.08   
oit    m       99.64   
       f        0.36   
ête    f       99.64   
       m        0.36   
cité   f       99.91   
       m        0.09   
mise   f       99.97   
       m        0.03   
uite   f       99.28   
       m        0.72   
c      m       97.82   
       f        2.18   
f      m       91.19   
       f        8.81   
ours   m       99.96   
       f        0.04   
adre   m       99.47   
       f        0.53   
port   m       99.95   
       f        0.05   
nne    f       99.59   
       m        0.41   
sité   f       99.99   
       m        0.01   
rtie   f       99.94   
       m        0.06   
ail    m       99.97   
       f        0.03   
ns     m       98.48   
       f        1.52   
tère   m       99.63   
       f        0.37   
nté    f       99.77   
       m        0.23   
p      m       99.06   
       f        0.94   
ande   f